# Check your background of numerical programming

This notebook is a quick recap of the main practical concepts of numerical programming, and introduces the Casadi module of the Pinocchio library:
- Pinocchio is obviously a library gathering the main rigid-body dynamics library.
- Casadi is an auto-differentiation tools, i.e. you can write symbolic expression and differentiate them. It is also wrapping the IpOpt solver, which is a quite robust and generic constraint-based optimization solver. You can then quickly write a problem with some symbolic expression for the cost and constraints, and Casadi will transfer it to IpOpt for solving it.
- The Pinocchio/Casadi module is a part of the Pinocchio solver written by Justin Carpentier which uses Casadi to unroll the algorithms of Pinocchio into symbolic expression, which then can be differentiated. It is a very powerful tool for initiation and prototyping, although it cannot scales to very large problems (in particular whole-body optimization problems) due to various technical problems that we can discuss.

## Definition of the toy problem studied here

We propose to explain how to define and solve inverse-geometry problems, i.e. problems where the main decision variable is a single robot configuration.

Inverse geometry is the problem of computing a configuration $q$ of a robot model that satisfies a set of 'tasks', which we formalize as error functions $e(q)$ that are zero when the task is achieved.

- $q \in \mathcal{C}$: configuration of the robot.
- $e_i(q) \in \mathbb{R}$: error function for task $i$. In this tutorial our error function is real valued.

Typical tasks for limbed robots include putting a foot/end-effector on a surface, moving the center of mass (CoM) to a target location, etc. For example, an error function for a foot position task could be:

$$
e(q) = \|p_\mathrm{foot}^* - p_\mathrm{foot}(q)\|
$$

In this tutorial, we will define a task function $e(q)$ for the end-effector pose of a manipulator and solve the inverse geometry problem

$$
\underset{q}{\mathrm{minimize}}\ e(q)
$$

using an off-the-shelf nonlinear optimizer that evaluates gradients by finite differences (slower), then a more efficient formulation using CasADi (faster).

## Before we start...

%load is a "magic" command of Jupyter. When we will invite you to think to a solution by yourselves, we will indicate it by the %do_not_load command. Feel free to transform it to a plain %load if you want the answer. This is define in the following import.

In [1]:
import magic_donotload

NB: as for all the tutorials, a magic command %do_not_load is introduced to hide the solutions to some questions. Change it for %load if you want to see (and execute) the solution.


Let's import everything we will need in this tutorial:

In [2]:
# %load tp1/generated/invgeom_imports
import time
import unittest
import casadi
import tp1.meshcat_shapes as meshcat_shapes
import example_robot_data as robex
import meshcat
import numpy as np
import pinocchio as pin
from numpy.linalg import norm
from scipy.optimize import fmin_bfgs

## Set up

Let us start by loading the UR5 robot model from `example-robot-data` and define its neutral configuration as follows:

In [6]:
# %load tp1/generated/invgeom_robot
robot = robex.load("ur10")
robot.q0 = np.array([0, -np.pi / 2, 0, 0, 0, 0])
model = robot.model
data = robot.data

We will also use the MeshCat visualizer to watch the optimization unfold. Let's set up a visualizer for our robot model:

In [7]:
# %load tp1/generated/invgeom_visualizer
viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model
)
robot.setVisualizer(viz, init=False)
viz.initViewer(open=False)
viz.loadViewerModel()
viz.display(robot.q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [13]:
viz.display(robot.q0*3)

In [8]:
viz.viewer.jupyter_cell()

## Part 1: BFGS with finite difference

In this first part, we will define an inverse geometry problem for the UR5 and use the BFGS optimizer from [`scipy.optimize.fmin_bfgs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_bfgs.html#scipy-optimize-fmin-bfgs) to solve it.

### Task target

Our task will be to move the end effector *frame* of the UR5 to a desired pose (pose := position + orientation). For this purpose, we define:

- The name of the corresponding frame, which is ``"tool0"`` for the UR5.
- The target pose of this frame, which we define by an affine transform ${}_{world} M_{target} \in SE(3)$ (also known as a "placement").

The robot will start from an arbitrary initial configuration (hint: to make the solver's life easier, avoid the singular $[0\ 0\ \ldots\ 0]$ configuration) $q_0$.

In [9]:
# %load tp1/generated/invgeom_task_params
tool_id = model.getFrameId("tool0")

transform_target_to_world = pin.SE3(
    pin.utils.rotate("x", np.pi / 4),
    np.array([-0.5, 0.1, 0.2]),
)

### Task error

For a given robot configuration $q$, the end effector will be located at ${}_{world} M_{tool}(q)$, which is a function of $q$ we can compute with Pinocchio's forward kinematics function. The goal of our task will be to make ${}_{world} M_{tool}(q)$ coincide with ${}_{world} M_{target}$, which we can define mathematically as: minimize the (Lie-group) difference ${}_{world} M_{target} \ominus {}_{world} M_{tool}(q)$, or equivalently:

$$
\underset{q}{\mathrm{minimize}} \ e(q) := \| {}_{world} M_{target} \ominus {}_{world} M_{tool}(q) \|
$$

Remember the definition of the $\ominus$ (left) Lie-group difference? No? Let's write it in Python anyway!

In [10]:
# %load tp1/generated/invgeom_error_function


def error(q: np.ndarray) -> float:
    """Function that the final configuration should minimize."""
    pin.framesForwardKinematics(model, data, q)
    transform_tool_to_world = data.oMf[tool_id]
    return norm(
        pin.log(transform_tool_to_world.inverse() * transform_target_to_world).vector
    )

### Sidetrack: visualization



In [11]:
# %load tp1/generated/invgeom_visualizer_callback
viewer = viz.viewer
meshcat_shapes.frame(viewer["target"], opacity=1.0)
meshcat_shapes.frame(viewer["current"], opacity=0.5)


def callback(q: np.ndarray):
    pin.framesForwardKinematics(model, data, q)
    transform_frame_to_world = data.oMf[tool_id]
    viewer["target"].set_transform(transform_target_to_world.np)
    viewer["current"].set_transform(transform_frame_to_world.np)
    viz.display(q)
    time.sleep(1e-1)

We are now ready to optimize for real!

### BFGS optimization

In [19]:
# %load tp1/generated/invgeom_fmin_bfgs
qguess = np.array([0.12, -2.2, -1.45, 1.82, -0.95, 0.17])
qopt = fmin_bfgs(error, qguess, callback=callback)

print(
    "The robot finally reached effector placement at\n",
    robot.placement(qopt, 6),
)

         Current function value: 0.000000
         Iterations: 70
         Function evaluations: 985
         Gradient evaluations: 139
The robot finally reached effector placement at
   R =
           1 -2.85321e-09 -8.64161e-09
-8.12807e-09    -0.707107    -0.707107
-4.09302e-09     0.707107    -0.707107
  p =     -0.5 0.165195 0.134805



If everything went well, our robot converged to a configuration $q$ that achieves the desired end effector placement, *i.e.* $e(q) \approx 0 \Leftrightarrow {}_{world} M_{target} \approx {{}_{world} M_{tool}}(q)$.

## Part 2: IPOPT with automatic differentiation

We wil now define our problem in [CasADi](https://web.casadi.org), a framework that allows us to formulate problems as computation graphs from which it can compute derivatives (notably first-order derivatives, *a.k.a.* gradients, and second-order derivatives, *a.k.a.* Hessians) automatically. This feature is called *automatic differentiation*.

### Modelling with CasADi

Pinocchio, in the version 3 we are using in today's tutorial, provides a `pinocchio.casadi` interface to represent configurations (such as $q \in \mathcal{C}$ or $M \in SE(3)$) in CasADi computation graphs, and differentiate them appropriately.

In [20]:
# %load tp1/generated/invgeom_casadi_imports
import casadi
from pinocchio import casadi as cpin

CasADi being designed for Euclidean variables, differentiating properly over manifolds is not as straightforward as representing everything as matrices and vectors in $\mathbb{R}^n$ and taking their usual derivatives. Pinocchio allows us to take derivatives over manifolds properly, without worrying about what happens below the surface. To be able to do that, we first need to redefine our model as a CasADi one:

In [21]:
# %load tp1/generated/invgeom_casadi_model
cmodel = cpin.Model(model)
cdata = cmodel.createData()

Our configuration variable $q$ is now a CasADi ``SX`` symbol of size $n_q$:

In [22]:
# %load tp1/generated/invgeom_cq
cq = casadi.SX.sym("q", model.nq, 1)

We complete the computation graph from ``cmodel`` to ``cq`` by calling the corresponding forward-kinematics function:

In [23]:
# %load tp1/generated/invgeom_casadi_fk
cpin.framesForwardKinematics(cmodel, cdata, cq)

In [37]:
cdata.oMf[tool_id]

SE3(SX(@1=cos(q_0), @2=4.89664e-12, @3=cos(q_1), @4=sin(q_1), @5=((@2*@3)-@4), @6=(@1*@5), @7=cos(q_2), @8=((@2*@4)+@3), @9=(@1*@8), @10=sin(q_2), @11=((@6*@7)-(@9*@10)), @12=cos(q_3), @13=sin(q_3), @14=((@2*@12)-@13), @15=((@6*@10)+(@9*@7)), @16=(@12+(@2*@13)), @17=((@11*@14)-(@15*@16)), @18=cos(q_4), @19=sin(q_0), @20=sin(q_4), @21=((@17*@18)-(@19*@20)), @22=cos(q_5), @23=((@2*@13)+@12), @24=((@2*@12)-@13), @25=((@11*@23)+(@15*@24)), @26=sin(q_5), @27=(@19*@5), @28=(@19*@8), @29=((@27*@7)-(@28*@10)), @30=((@27*@10)+(@28*@7)), @31=((@29*@14)-(@30*@16)), @32=((@31*@18)+(@1*@20)), @33=((@29*@23)+(@30*@24)), @34=(@3+(@2*@4)), @35=((@2*@3)-@4), @36=((@34*@7)+(@35*@10)), @37=((@35*@7)-(@34*@10)), @38=((@36*@14)+(@37*@16)), @39=(@38*@18), @40=((@37*@24)-(@36*@23)), @41=0, @42=((@17*@20)+(@19*@18)), @43=((@21*@26)+(@25*@22)), @44=((@1*@18)-(@31*@20)), @45=((@32*@26)+(@33*@22)), @46=(@38*@20), @47=((@40*@22)-(@39*@26)), @48=0.612, @49=-0.1719, @50=0.220941, @51=0.5723, @52=0.1149, @53=0.1157,

### Task error

We define the same task error as above:

$$
\underset{q}{\mathrm{minimize}} \ e(q) := \| {}_{world} M_{target} \ominus {}_{world} M_{tool}(q) \|
$$

But this time as a CasADi function:

In [ ]:
# %load tp1/generated/invgeom_casadi_error
error_tool = casadi.Function(
    "etool",
    [cq],
    [
        cpin.log6(
            cdata.oMf[tool_id].inverse() * cpin.SE3(transform_target_to_world)
        ).vector
    ],
)

In [39]:
cpin.log6(cpin.SE3(transform_target_to_world)).vector

SX(@1=0, [-0.5, 0.173346, 0.150342, 0.785398, @1, @1])

Finally, we define a computation graph from our optimization variable to our error function:

In [32]:
# %load tp1/generated/invgeom_casadi_computation_graph
opti = casadi.Opti()
var_q = opti.variable(model.nq)
totalcost = casadi.sumsqr(error_tool(var_q))

In [35]:
var_q.shape

(6, 1)

### IPOPT optimization

In [57]:
# %load tp1/generated/invgeom_ipopt
opti.minimize(totalcost)
opti.solver("ipopt")  # select the backend solver
opti.callback(lambda i: callback(opti.debug.value(var_q)))
opti.set_initial(var_q,np.ones(6)*3)

# Caution: in case the solver does not converge, we are picking the candidate values
# at the last iteration in opti.debug, and they are NO guarantee of what they mean.
try:
    sol = opti.solve_limited()
    sol_q = opti.value(var_q)
except:
    print("ERROR in convergence, plotting debug info.")
    sol_q = opti.debug.value(var_q)

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

We can check that the solver converged to the same solution:

In [44]:
# %load tp1/generated/invgeom_check_final_placement
pin.framesForwardKinematics(model,data,sol_q)

print(
    "The robot finally reached effector placement at\n",
    #robot.placement(sol_q, 6),
    data.oMf[tool_id], transform_target_to_world
)

The robot finally reached effector placement at
   R =
           1  4.44089e-16 -1.66533e-16
-2.77556e-16     0.707107    -0.707107
-1.11022e-16     0.707107     0.707107
  p = -0.5  0.1  0.2
   R =
        1         0         0
        0  0.707107 -0.707107
        0  0.707107  0.707107
  p = -0.5  0.1  0.2



In [42]:
opti.value(error_tool(var_q))

array([ 3.88578059e-15,  3.27515792e-15, -2.24820162e-15, -4.44089210e-16,
        1.66533454e-16,  2.74766180e-16])

To conclude, count the number of iterations and number of function/gradient evaluations. This is automatic differentiation 🤠

In [28]:
pin.framesForwardKinematics(model,data,sol_q)

In [30]:
print(data.oMf[tool_id])

  R =
           1  4.44089e-16 -1.66533e-16
-2.77556e-16     0.707107    -0.707107
-1.11022e-16     0.707107     0.707107
  p = -0.5  0.1  0.2



In [31]:
print(data.oMi[6])

  R =
           1 -1.66533e-16 -4.44089e-16
-2.77556e-16    -0.707107    -0.707107
-1.11022e-16     0.707107    -0.707107
  p =     -0.5 0.165195 0.134805



In [51]:
viz.display(qopt)

In [52]:
viz.display(sol_q)